In [1]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('Awake_Angad.csv')
df2 = pd.read_csv('Awake_Chandra.csv')
df3 = pd.read_csv('Awake_Prottasha.csv')
df10 = pd.read_csv('Awake_chandra1.csv')
df13 = pd.read_csv('Awake_Chandra3.csv')

df4=pd.read_csv('relax_angad.csv')
df5=pd.read_csv('relax_chandra.csv')
df6=pd.read_csv('relax_Prottasha.csv')
df11=pd.read_csv('relax_chandra2.csv')
df14=pd.read_csv('relax_chandra3.csv')

df7=pd.read_csv('Invalid.csv')
df8=pd.read_csv('Invalid2.csv')
df9=pd.read_csv('Invalid3.csv')
df12=pd.read_csv('Invalid4.csv')

df1['Label'] = 1
df2['Label'] = 1
df3['Label'] = 1
df10['Label'] = 1
df13['Label'] = 1

df4['Label'] = 0
df5['Label'] = 0
df6['Label'] = 0
df11['Label'] = 0
df14['Label'] = 0

df7['Label'] = 2
df8['Label'] = 2
df9['Label'] = 2
df12['Label'] = 2



Awake_df = pd.concat([df1, df2, df3,df10,df13], ignore_index=True)
Relax_df = pd.concat([df4,df5,df6,df11,df14], ignore_index=True)
Invalid_df=pd.concat([df7,df8,df9,df12], ignore_index=True)

combined_df = pd.concat([Awake_df,Relax_df,Invalid_df], ignore_index=True)

In [3]:
print(Awake_df['Alpha_Metric'])

0     -0.428147
1     -0.156850
2     -0.045981
3      0.036202
4      0.107916
         ...   
750    0.461037
751    0.479089
752    0.478905
753    0.470106
754    0.456194
Name: Alpha_Metric, Length: 755, dtype: float64


In [4]:
X = combined_df[['Alpha_Metric', 'Beta_Metric']]

In [5]:
X.shape

(2114, 2)

In [6]:
y=combined_df['Label']

In [7]:
y.shape

(2114,)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVC(kernel='rbf')
svm_model.fit(X_train_scaled, y_train)


SVC()

In [9]:

predictions = svm_model.predict(X_test_scaled)
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      0.53      0.69       138
           1       0.65      0.97      0.77       158
           2       0.93      0.83      0.88       127

    accuracy                           0.78       423
   macro avg       0.86      0.77      0.78       423
weighted avg       0.85      0.78      0.78       423



In [10]:
import socket
import time  # Import the time module

# Raspberry Pi's IP address and port
RASPBERRY_PI_IP = '10.0.0.246'
PORT = 65432

# Function to start a socket
def start_socket():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((RASPBERRY_PI_IP, PORT))
    return client_socket

# Function to close a socket
def close_socket(client_socket):
    client_socket.close()
    print("Socket closed")

# Function to send commands to the Raspberry Pi
def send_command(client_socket, command):
    client_socket.sendall(command.encode('utf-8'))
    #print(f"Command sent to Raspberry Pi: {command}")
    time.sleep(0.1)




In [37]:
import time
import numpy as np
from pylsl import StreamInlet, resolve_byprop
import utils
import warnings
import serial

# Suppress all warnings
warnings.filterwarnings('ignore')

class Band:
    Delta = 0
    Theta = 1
    Alpha = 2
    Beta = 3

# Constants
SAMPLE_RATE = 500
BAUD_RATE = 115200
BUFFER_SIZE = 128

BUFFER_LENGTH = 5
EPOCH_LENGTH = 1
OVERLAP_LENGTH = 0.8
SHIFT_LENGTH = EPOCH_LENGTH - OVERLAP_LENGTH
INDEX_CHANNEL = [0]
SAMPLE_COUNTER = 0
PREDICTIONS = []
last_good_EMG_data = None
# Configure serial port
ser = serial.Serial(
    port='/dev/tty.usbmodem11301',  # Adjust as per your setup
    baudrate=BAUD_RATE,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout=1
)

def read_serial():
    """
    Reads a line from the serial port and returns the parsed values as a list.
    """
    try:
        if ser.in_waiting > 0:
            line = ser.readline().decode('utf-8').strip()
            values = list(map(int, line.split(',')))
            return values  # Return parsed values as a list
    except (UnicodeDecodeError, ValueError):
        pass  # Ignore faulty readings
    return []

if __name__ == "__main__":
    print('Looking for an EEG stream...')
    streams = resolve_byprop('type', 'EEG', timeout=2)
    if not streams:
        raise RuntimeError("Can't find EEG stream.")

    print("Start acquiring data")
    inlet = StreamInlet(streams[0], max_chunklen=12)
    eeg_time_correction = inlet.time_correction()

    try:
        time.sleep(2)

        info = inlet.info()
        fs = int(info.nominal_srate())

        eeg_buffer = np.zeros((int(fs * BUFFER_LENGTH), 1))
        filter_state = None
        n_win_test = int(np.floor((BUFFER_LENGTH - EPOCH_LENGTH) / SHIFT_LENGTH + 1))
        band_buffer = np.zeros((n_win_test, 4))
        START_TIME = time.time()
        print('Press Ctrl-C in the console to break the while loop.')

        client_socket = start_socket()
        while True:
            eeg_data, timestamp = inlet.pull_chunk(timeout=1, max_samples=int(SHIFT_LENGTH * fs))
            ch_data = np.array(eeg_data)[:, INDEX_CHANNEL]
            eeg_buffer, filter_state = utils.update_buffer(eeg_buffer, ch_data, notch=True, filter_state=filter_state)

            data_epoch = utils.get_last_data(eeg_buffer, EPOCH_LENGTH * fs)
            band_powers = utils.compute_band_powers(data_epoch, fs)
            band_buffer, _ = utils.update_buffer(band_buffer, np.asarray([band_powers]))
            smooth_band_powers = np.mean(band_buffer, axis=0)

            if time.time() - START_TIME >= 10:
                SAMPLE_COUNTER += 1  # Increment sample counter

            alpha_metric = smooth_band_powers[Band.Alpha] / smooth_band_powers[Band.Delta]
            beta_metric = smooth_band_powers[Band.Beta] / smooth_band_powers[Band.Theta]
            features = np.array([[alpha_metric, beta_metric]])
            features_scaled = scaler.transform(features)
            y_pred = svm_model.predict(features_scaled)
            EMG_data = read_serial()
            if EMG_data == None or len(EMG_data) == 0:
                EMG_data=last_good_EMG_data
            else:
                last_good_EMG_data=EMG_data
            print(EMG_data)
            if time.time() - START_TIME >= 10 and all(EMG_data[i] < 10 for i in [1, 3]):
                PREDICTIONS.append(y_pred[0])
                if SAMPLE_COUNTER % 5 == 0:
                    most_frequent_prediction = np.bincount(PREDICTIONS).argmax()
                    if most_frequent_prediction == 0:
                        #print('Move Back')
                        send_command(client_socket, "backward")
                    elif most_frequent_prediction == 1:
                        #print('Move Forward')
                        send_command(client_socket, "forward")
                    elif most_frequent_prediction == 2:
                        #print('Invalid')
                        send_command(client_socket, "stop")
                    PREDICTIONS.clear()
            elif time.time() - START_TIME >= 10 and any(EMG_data[i] >= 10 for i in [1, 3]):
                if EMG_data[1] >= 10 and EMG_data[3] >= 10:
                    # If both thresholds are met, stop the car.
                    send_command(client_socket, "stop")
                elif EMG_data[1] >= 10:
                    # If only the right threshold is met, turn right.
                    send_command(client_socket, "right")
                elif EMG_data[3] >= 10:
                    # If only the left threshold is met, turn left.
                    send_command(client_socket, "left")

    except KeyboardInterrupt:
        print('Closing!')
        close_socket(client_socket)
        ser.close()


Looking for an EEG stream...
Start acquiring data
Press Ctrl-C in the console to break the while loop.
[0, 9, 0, 9]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 3, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 2]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 7, 0, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 17, 0, 0]
[0, 36, 0, 0]
[0, 30, 0, 0]
[0, 13, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 2, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 2, 0, 0]
[0, 2, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 2, 0, 1]
[0, 13, 0, 5]
[0, 13, 0, 5]
[0, 16, 0, 6]
[0, 13, 0, 5]
[0, 10, 0, 5]
[0, 8, 0, 4]
[0, 8, 0, 4]
[0, 

BrokenPipeError: [Errno 32] Broken pipe